目的：对每个题目，评估他的难度等级，并且对不同难度等级的构造不同的分位数？这个看起来是一个比较直接的办法
此外就是预测reasoning的长度，这个看起来是一种更灵活的方式
或者把两个东西结合起来
比如说实际当中，可以先用随机森林来预测，然后结合类别（比如说可以分成五类……）


——————————
需要做的事情：
1. 完成conditional CP的代码（这个比较重要）（可以用他们的python包来预测）
2. 完善预测的代码，用随机森林怎么才能用的更好一点点
3. test 在新的数据上有多大的正确率，包括他的



-----
如何真正的解决实际当中reasoning的问题呢？？？一般来说人们用conformal prediction是在提供一个置信区间
更新overleaf，目前觉得用conditional 是一个不错的方法，但是实际中是

In [5]:
import json
import re
import time
from tqdm import tqdm
from openai import OpenAI
from plevel_tools.level_prompt import judge_level


input_path = "D:\math\A-reasoning_demo\judged_datasets\HuggingFaceH4\MATH-500.jsonl"
output_path = "D:\math\A-reasoning_demo\judged_datasets\HuggingFaceH4\MATH-500_with_levels.jsonl"

client = OpenAI(
    api_key="sk-28d3e9b60f9f43a0a958ae132b7fcc2f",
    base_url="https://api.deepseek.com"
)


def extract_level(text):
    match = re.search(r"\\boxed\{(\d)\}", text)
    if match:
        return int(match.group(1))
    return None

def query_with_retry(prompt, retries=3, delay=5):
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant"},
                    {"role": "user", "content": prompt}
                ],
                stream=False
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"[Retry {attempt+1}/{retries}] Error: {e}")
            time.sleep(delay)
    return None 


with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in tqdm(infile, desc="Processing problems"):
        data = json.loads(line)
        problem = data.get("problem", "")

        prompt = judge_level.format(problem=problem)
        reply = query_with_retry(prompt, retries=3, delay=5)

        if reply is not None:
            level = extract_level(reply)
            data["pred_level"] = level
        else:
            data["pred_level"] = None

        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")


Processing problems: 138it [12:00,  5.22s/it]


In [ ]:
data = []

i = 0
with open (output_path,'r',encoding='utf-8') as file:
    for line in file:
        a_data = 



In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from conditionalconformal.synthetic_data import generate_cqr_data, indicator_matrix
from conditionalconformal import CondConf

problem_calib = []
l_calib = []
problem_test = []
l_test = []
i=0


with open (output_path,'r',encoding='utf-8') as file:
    for line in file:
        i+=1
        if(i<=100):
            data = json.loads(line)
            problem_calib.append({"problem":data.get("problem", ""),"level":data.get("pred_level","")})
            l_calib.append(data.get("min_valid_length",""))
        else:
            data = json.loads(line)
            problem_test.append({"problem":data.get("problem", ""),"level":data.get("pred_level","")})
            l_test.append(data.get("min_valid_length",""))
alpha = 0.1

score_fn = lambda x,y: y
score_inv_fn_lb = lambda s, x : [s, np.inf]

eps = 1
disc = np.arange(0.5, 5 + eps, eps) 
def phi_fn_groups(x):
    levels = [data["level"] for data in x]
    return indicator_matrix(levels, disc)

cond_conf = CondConf(score_fn,phi_fn_groups,infinite_params={})
cond_conf.setup_problem(x_calib=problem_calib,y_calib=l_calib)